# Notebook 1: Scraper/HTML Parser

##### Please refer to the Python Requirements and Installation Guide pdf 

##### Purpose: The purpose of the notebook is to scrape Kununu for companies the following companies: 

1. __Bechtle__ : https://www.kununu.com/de/bechtle/kommentare
2. __SAP__: https://www.kununu.com/de/sap/kommentare
3. __IBM__: https://www.kununu.com/de/ibm-deutschland/kommentare
4. __T-Systems__: https://www.kununu.com/de/t-systems/kommentare 
5. __Computacenter__: https://www.kununu.com/de/computacenter/kommentare
6. __Swisscom__: https://www.kununu.com/ch/swisscom/kommentare
7. __Cancom__: https://www.kununu.com/de/cancom/kommentare
8. __Adesso__: https://www.kununu.com/de/adesso/kommentare
9. __Capegemini__: https://www.kununu.com/de/capgemini/kommentare
10. __Fujitsu__: https://www.kununu.com/de/fujitsu-technology-solutions/kommentare
11. __Dell__: https://www.kununu.com/de/dell5/kommentare


#### Additional Python Libraries required: 
 1.   __Selenium__ <br> 
      Link: https://pypi.org/project/selenium/ <br>
      pip install -U selenium <br>
 2. __Beautiful Soup__<br>
      Link: https://pypi.org/project/beautifulsoup4/ <br>
      pip install beautifulsoup4 <br>
 3. __Transformers__ <br>
      Link: https://pypi.org/project/transformers/ <br>
      pip install transformers
      
    

# Imports required within the notebook:

###### Please find the first_notebook_requirements.txt file within the folder. 

In [1]:
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  0


In [1]:
# Importing the required packages for this notebook: 


import os
from bs4 import BeautifulSoup
import re
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 30)
import json
import numpy as np
import pandas as pd
import requests
from bs4 import BeautifulSoup
import time
import json
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
import re
import requests
from bs4 import BeautifulSoup
from timeit import default_timer as timer
from tqdm import tqdm
import time
import sys
import warnings
warnings.filterwarnings('always', message='Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length.')
warnings.filterwarnings('always', message= "UserWarning: Neither `max_length` nor `max_new_tokens` has been set, `max_length` will default to 200 (`self.config.max_length)")
warnings.simplefilter(action='always', category = FutureWarning)
warnings.filterwarnings('always')
warnings.simplefilter('ignore')

# 1. Scraping Kununu:

#### First step: Get the url link of the Kununu page that will be scraped. Below are the companies we used for our analysis. 

In [ ]:
#variable containing the url of each of the companies that we would like to scrape: 
url_sap = 'https://www.kununu.com/de/sap/kommentare'
url_ibm = "https://www.kununu.com/de/ibm-deutschland/kommentare"
url_bechtle = "https://www.kununu.com/de/bechtle/kommentare"
url_t_systems = "https://www.kununu.com/de/t-systems/kommentare"
url_computacenter = "https://www.kununu.com/de/computacenter/kommentare"
url_swisscom = "https://www.kununu.com/ch/swisscom/kommentare"
url_cancom = "https://www.kununu.com/de/cancom/kommentare"
url_adesso = "https://www.kununu.com/de/adesso/kommentare"
url_capgemini = "https://www.kununu.com/de/capgemini/kommentare"
url_fujitsu = "https://www.kununu.com/de/fujitsu-technology-solutions/kommentare"
url_dell = "https://www.kununu.com/de/dell5/kommentare"

#### The following function takes in the url and scrapes the data saving it as an html parser:

In [ ]:
def scraper(url, html_file_name):
    """
    Purpose: The purspose of the function is to take in input as url and scrape the desired url. The function
    then return a html file with all the scraped data. 
    Parameters:
         url:  The desired website url to be scraped
         html_file_name: The desired name of the html file name for the scraped url
    Return: returns a string invoking that the scraping of the desired company is completed. 
    """
    
    # chromedrivermanager is required. It is an automated browser that opens and is controlled by the code within.
    from webdriver_manager.chrome import ChromeDriverManager
    driver = webdriver.Chrome(ChromeDriverManager().install()) 
    print("function started")
    time.sleep(3)
    url = url
    #driver.get(url) is utilised to load the required url into the automated web browser. 
    driver.get(url) 
# The xpath below is for accepting cookies that first appear when we enter the url. 
    driver.find_element('xpath','//*[@id="consent-accept-button"]').click()  
    time.sleep(5) # this is given to wait for 5 seconds to accept the cookies
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    # index__tabCount__3oL0k class contains the number of reviews generated/written for the company
    review_cnt = soup.find('span', class_= "index__tabCount__3oL0k")
    print(review_cnt)
    review_cnt_int = int(re.sub(r'[^\w]', '', review_cnt.text))
#  The clicks variable below is used to store the number of clicks that will be required to load all the reviews. 
#  We give an additional of 20 clicks which computationally would not be clicked, once the break statement is implemented.
    clicks = round(review_cnt_int/10) + 20 
    print(clicks)
    print("starting full scraper")
    j = 10 # current count of clicks 
    for i in range(0, clicks):
        try:
            driver.find_element('xpath','//*[@id="reviews-read-more-cta"]').click()
            print(j)
            print(f'{i}th iteration running')
            print('***************************')
            # loading period of 4 seconds given to load all the clicks 
            time.sleep(4)
        except:
            break
        j =j+10
# Once all the reviews have been reviewed, we parse the webpage using BeautifulSoup
    print("parsing html file")
    soup = BeautifulSoup(driver.page_source, 'html.parser')
# Identify the class containing each review and we find all of them
    all_reviews = soup.find_all('div', class_ = 'index__reviewBlock__27gnB')
    print(len(all_reviews))
    print("saving html file")
# saved html outpout into directory:
    with open(html_file_name, "w", encoding = 'utf-8') as file:
        # prettify the soup object and convert it into a string
        file.write(str(soup.prettify()))

    return("scraping of", html_file_name, "done and saved to directory")


#### The next step was to call the function and pass each url and save the html file: 

The function call has been commennted. Please uncomment in order to run the scraper and save the parsed html file
into directory. The scraping takes approximately 15 minutes per company.

In [4]:
# PLEASE UNCOMMENT ANY OF THEM TO TEST THE SCRAPER: 
# scraper(url_computacenter, "url_computacenter.html")
# scraper(url_sap ,'SAP_output (1).html')
# scraper(url_ibm , "IBM_output.html")
# scraper(url_bechtle,"bechtle_output.html")
# scraper(url_t_systems ,'url_t_systems_output.html')
# scraper(url_computacenter , "url_computacenter.html")
# scraper(url_swisscom , "url_swisscom.html")
# scraper(url_cancom , "url_cancom.html")
# scraper(url_adesso , "url_adesso.html")
# scraper(url_capgemini,"url_capegemini.html")
# scraper(url_fujitsu,"url_fujitsu.html")
# scraper(url_dell,"url_dell.html"this is a 

# 2. HTML Parser 

#### The next step is to convert the html file into structured dataframes and save them as csv files. We extract information following information which also becomes our column names in the dataframe: 

1. __review_date__ <br>
2. __review_title__ <br>
3. __review_recommendation__ <br>
4. __review_rating__ <br>
5. __review_employee_info__ <br>
6. __Arbeitsatmosphäre_star__ <br>
7. __Arbeitsatmosphäre_plain_text__ <br>
8. __Work-Life-Balance_star__ <br>
9. __Work-Life-Balance_plain_text__ <br>
10. __Kollegenzusammenhalt_star__ <br>
11. __Kollegenzusammenhalt_plain_text__ <br>
12. __Vorgesetztenverhalten_star__ <br> 
13. __Vorgesetztenverhalten_plain_text__ <br>
14. __Kommunikation_star__ <br>
15. __Kommunikation_plain_text__ <br>
16. __Gehalt/Sozialleistungen_star__ <br> 
17. __Gehalt/Sozialleistungen_plain_text__ <br>
18. __Gut am Arbeitgeber finde ich_st__ <br>
19. __Gut am Arbeitgeber finde ich_plain_text__ <br>
20. __Schlecht am Arbeitgeber finde ich_star__ <br>
21. __Schlecht am Arbeitgeber finde ich_plain_text__ <br>
22. __Verbesserungsvorschläge_star__ <br> 
23. __Verbesserungsvorschläge_plain_text__ <br> 
24. __Image_star__ <br> 
25. __Image_plain_text__ <br> 
26. __Karriere/Weiterbildung_star__ <br>
27. __Karriere/Weiterbildung_plain_text__ <br>
28. __Umwelt-/Sozialbewusstsein_plain_text__ <br> 
29. __Umgang mit älteren Kollegen_star__ <br> 
30. __Umgang mit älteren Kollegen_plain_text__ <br> 
31. __Arbeitsbedingungen_star__ <br> 
32. __Arbeitsbedingungen_plain_text__ <br> 
33. __Gleichberechtigung_star__ <br> 
34. __Gleichberechtigung_plain_text__ <br> 
35. __Interessante Aufgaben_star__ <br> 
36. __Arbeitszeiten_star__ <br> 
37. __Arbeitszeiten_plain_text__ <br> 
38. __Ausbildungsvergütung_star__ <br>
39. __Ausbildungsvergütung_plain_text__ <br> 
40. __Die Ausbilder_star__ <br> 
41. __Die Ausbilder_plain_text__ <br> 
42. __Aufgaben/Tätigkeiten_star__ <br> 
43. __Aufgaben/Tätigkeiten_plain_text__ <br> 
44. __Variation_star__ <br> 
45. __Variation_plain_text__ <br> 
46. __Respekt_star__ <br> 
47. __Respekt_plain_text__ <br> 
48. __Karrierechancen_star__ <br> 
49. __Karrierechancen_plain_text__ <br> 
50. __Spaßfaktor_star__ <br> 
51. __Spaßfaktor_plain_text__ <br> 
52. __Wie kann dich dein Arbeitgeber im Umgang mit der Corona-Situation noch besser unterstützen?_star__ <br> 
53. __Wie kann dich dein Arbeitgeber im Umgang mit der Corona-Situation noch besser unterstützen?_plain_text__ <br> 
54. __Wofür möchtest du deinen Arbeitgeber im Umgang mit der Corona-Situation loben?_star__ <br> 
55. __Wofür möchtest du deinen Arbeitgeber im Umgang mit der Corona-Situation loben?_plain_text__ <br> 
56. __Was macht dein Arbeitgeber im Umgang mit der Corona-Situation nicht gut?_star__ <br> 
57. __Was macht dein Arbeitgeber im Umgang mit der Corona-Situation nicht gut?_plain_text__ <br> 
58. __Wo siehst du Chancen für deinen Arbeitgeber mit der Corona-Situation besser umzugehen?_star__ <br> 
59. __Wo siehst du Chancen für deinen Arbeitgeber mit der Corona-Situation besser umzugehen?_plain_text__ <br>

#### Such information comprises of both the stars and the textual reviews that ex-employees have written on Kununu. 

#### The next step is to traverse through the html file and extract the information above. For this, we have created the function below. 

In [4]:
def create_soup(path_to_html: str):
    
    """
    Purpose: The purpose the function is to take as input the path where the html file from the above "scraper" function
    is stored and open it as a html file. 
    Paramters: 
        path_to_html: document path to where the website parsed was saved
    
    Return: The function returns a dataframe for each of the company html files that are passed to this function.
    """
    
    
    with open(path_to_html, encoding="utf8") as f:
        output_soup = BeautifulSoup(f, 'html.parser')
    return output_soup.find_all(attrs = 'div', class_ = 'index__reviewBlock__27gnB')

def create_dictionaries(review_soup):
    '''
    Purpose: The purpose of the function is to create a dictionary for each of the companies. The key being the main
    topic of that the former employee is talking about and the value represents the stars and textual reviews that they 
    have left for the company. 
    
    Parameter: 
        review_soup: Contains the companies html file 
    Return: Returns a dictionary. The key representing the main topic for and the value represents the review that the 
    former employee has left behind for the company. 
    '''
    dictionary_output = {}
    i = 0
    for review in review_soup:
        dictionary_output[f'review_{i}'] = {}
        # Review header information 
        review_header = {}
        try:
            review_header['review_date'] = review.find(attrs = 'time', class_ = 'p-tiny-regular text-dark-63').get('datetime')
        except:
            review_header['review_date'] = np.NaN
        try:
            review_header['review_title'] = review.find(attrs = 'h3', class_ = 'index__title__2uQec h3-semibold').text
            review_header['review_title'] = review_header['review_title'].strip()
        except:
            review_header['review_title'] = np.NaN
        try:
            review_header['review_recommendation'] = review.find(attrs = 'span', class_ = 'p-tiny-bold').text
            review_header['review_recommendation'] = review_header['review_recommendation'].strip()
        except:
            review_header['review_recommendation'] = np.NaN
        try:
            review_header['review_rating'] = review.find(attrs = 'span', class_ = 'h3-semibold index__score__16yy9').text
            review_header['review_rating'] = review_header['review_rating'].strip()
            review_header['review_rating'] = float(review_header['review_rating'].replace(',', '.'))
        except:
            review_header['review_rating'] = np.NaN
        try:
            review_header['review_employee_info'] = review.find(attrs = 'b', class_ = 'index__employmentInfoBlock__27ro4 p-tiny-regular').text
            review_header['review_employee_info'] = review_header['review_employee_info'].replace('\n', ' ')
            review_header['review_employee_info'] = re.sub("\s\s+", " ", review_header['review_employee_info'])
        except:
            review_header['review_employee_info'] = np.NaN
        dictionary_output[f'review_{i}'].update(review_header)
#       Review body information: Need to find all tags belonging to index__factor__3Z15R p-base-regular class as
#       few features are placed within the same class
        review_body_list = review.find_all(attrs = 'div', class_ = 'index__factor__3Z15R p-base-regular')
        for body_item in review_body_list:
            body_title = body_item.find(attrs = 'h4', class_ = 'index__title__W4hOp').text
            body_title = body_title.strip()
            try:
                # try to find stars that have been left in the reviews 
                body_item_star = float(body_item.find(attrs = 'span', class_ = 'index__stars__2ads4 index__medium__1wpWb index__stars__3lgvx').get('data-score'))
            except:
                # if the stars are not present, then place NaN as a value 
                body_item_star = np.NaN
            try:
#                 Since sometime the feature does not contain any text, we placed a try and except in order to 
#                 retrieve the information if there is any that is present
                body_item_plain_text = body_item.find(attrs = 'p', class_ = 'index__plainText__lgNCM').text
                body_item_plain_text = body_item_plain_text.replace('\n', ' ')
                body_item_plain_text = re.sub('\s\s+', ' ', body_item_plain_text)
            except:
                body_item_plain_text = np.NaN
            dictionary_output[f'review_{i}'][body_title + '_star'] = body_item_star
            dictionary_output[f'review_{i}'][body_title + '_plain_text'] = body_item_plain_text
        i += 1
    # creates a dictionary for with key-value pairs for each company 
    return (dictionary_output)

def create_df(review_dict: dict, csv_output_name: str):
    '''
    Purpose: The purpose of this function is to convert the dictionary of each of the companies into a dataframe.
    
    Parameter: 
        review_dict: Takes input the dictionary for each of the companies. 
        csv_output_name: The desired name of the csv file containing the dataframe with all the textual information.
    '''
    df = pd.DataFrame.from_dict(review_dict).transpose()
    # saves the csv in the directory 
    df.to_csv(f'{csv_output_name}.csv', sep = '\t', encoding = 'utf-8', index = True)
    
    
def send_untraslated_csv(path, output_name):
    '''
    Purpose: The purpose of the function is to call all the functions above in this block. 
    
    Paramters: 
        path: The path where the html file is stored. 
        output_name: The name of the csv file that contains the dataframe. 
    '''

    review_soup = create_soup(path)
    review_soup = create_dictionaries(review_soup)
    create_df(review_dict = review_soup, csv_output_name = output_name)


### Run the below code to save the csv file into your directory of choice. Pass in the path of where you want the document to be saved and its corresponding name. 

#### We have provided a sample of how the function needs to be called. The same function had been called 11 times to derive csv files(dataframes) for each of the companies. 

In [5]:
# PLEASE PROVIDE YOUR OWN PATH AND NAME OF DESIRED CSV FILE: 
# Format of calling the function: send_untraslated_csv(path, csv_file_name)
# Here for the path, we provide the path that contains the html file. The html output can be found in the "HTML_scraped_output" directory/folder. 
# For instance, here, to scrape Cancom, we provided the path to the cancom html file generated from the above code. The second 
# argument refers to the csv file which will then be saved on the working directory. 
# In our submitted codes and results, the output dataframe/csv can be found in "output_dictionaries_and_raw_csv" folder/directory 
send_untraslated_csv(r"C:\Users\jdzuc\OneDrive\Frankfurt School Courses\Third Semester\Strategic Management\Final Project\Strategy Final Project\HTML_scraped_output\url_cancom.html", "cancom_german_reviews")

# 3. German to English Translation

### Please be aware that to execute the below function, one should have a GPU available, to increase the computational speed of the translation. Otherwise, the CPU will be used which results in a translation time of 3 hours per company csv. The below cell, checks if a GPU is detected. 

### To translate the dataframes, FairSeqMachineTranslation from Facebook was used, please run the below cell to import the model. 

In [5]:
from transformers import FSMTForConditionalGeneration, FSMTTokenizer
mname = "facebook/wmt19-de-en" # transformer utilzed for tokenization and decoding
tokenizer = FSMTTokenizer.from_pretrained(mname)
model = FSMTForConditionalGeneration.from_pretrained(mname)
# we used GPU for this task.

In [6]:
class translation:
    '''
    Purpose: The purpose of this class is to take input as the dataframe that had been created from the above functions. 
    The dataframe from above has all text in german. The aim of this class is to translate all the text from the 
    columns containing text into English.
    '''

    def __init__(self, df): 
        
        '''
        df: Dataframe that needs translation
        '''
        self.df = df
        self.translated_df = pd.DataFrame()
        self.translate_col_list = ['_title', '_recommendation', '_info', '_text']
        self.all_cols= list(self.df.columns)

    def lets_translate(self, text:str):
        '''
        Translation of columns
        '''
        input = text
        try:
            input_ids = tokenizer.encode(input, return_tensors="pt", max_length= 200000, truncation= True)# if gpu is not available please delete ".to(device)"
            outputs = model.generate(input_ids) # if gpu is not available please delete ".to(device)"
            decoded = tokenizer.decode(outputs[0], skip_special_tokens=True)

        except:
            return " "
        return decoded

    def get_list_cols(self):
        '''
        Gets the list of columns that need translation
        '''
        columns_to_translate = []
        for item in self.all_cols:
            if item[item.rfind('_'):] in self.translate_col_list:
                columns_to_translate.append(item)

        return columns_to_translate

    def execute(self):
        '''
         Executes everything and returns the dataframe that has been translated.
        '''
        for item in tqdm(self.all_cols):
            if item not in self.get_list_cols():
                self.translated_df[item]= self.df[item]
            else: 
                self.translated_df[item] = self.df[item].apply(lambda x: self.lets_translate(x))
        return self.translated_df


In [7]:
def csv_translated(path, file_name ):
    '''
    Purpose: The purpose of this function is to execute the class above, retrieve the translated dataframe and save it. 
    
    Parameters: 
        path: The path where the german version of text is placed. 
        file_name: The name of the desired translated csv file. 
    '''
    df = pd.read_csv(path, sep = '\t').rename(columns= {"Unnamed: 0": "review_idx"})
    trans_obj = translation(df)
    df_2 = trans_obj.execute()
    df_2.to_csv()
    df_2.to_csv(file_name, sep = '\t', encoding = 'utf-8', index = True)

### Run the below cell to import the csv file, by giving in the path and the name of the file. It will then translate the csv file and send back the translated dataframe. 

#### The csv_translated has been called 11 time for each of the companies. A sample has been provided below for the same. 

In [ ]:
# Format of the function call: 
# csv_translated(path to to untranslated file which can be found in the directory- "output_dictionaries_and_raw_csv", name of the translated file)
# The translated output/file will then be saved to your working directory and the results of the translation can be "found translated_csvs_folder" folder/directory
# For instance, here, we use the untranslated sap_reviews.csv to then translate and save into working directory as sap_translated.csv
csv_translated(r"C:\Users\jdzuc\OneDrive\Frankfurt School Courses\Third Semester\Strategic Management\Final Project\Strategy Final Project\output_dictionaries_and_raw_csv\sap_reviews.csv", f"sap_translated.csv")

#### End of Notebook
##### Next: The next notebook to utilise is 02_sentiment_analysis.ipynb